<a href="https://colab.research.google.com/github/cmattosr/imersao-ai-alura/blob/main/Aula05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Google SDK installation
!pip install -q -U google-generativeai

In [6]:
# Import the Python SDK
import google.generativeai as genai
from google.colab import userdata

import pandas as pd
import numpy as np

api_key = userdata.get("GEMINI_SECRET_KEY")
genai.configure(api_key=api_key)

In [7]:
for model in genai.list_models():
  if "embedContent" in model.supported_generation_methods:
    print(model.name)

models/embedding-001
models/text-embedding-004


In [8]:
title = "A próxima geraçãi de IA para desenvolvedores e Google workspace"
sample_text = ("Título: A próxima geraçãi de IA para desenvolvedores e Google workspace"
"\n"
"Artigo completo: \n"
"\n"
"Gemini API & Google AI Studio: Uma maneira acessível de explorar e criar protótipos com aplicações de IA generativa")

embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")

print(embeddings)

{'embedding': [0.049918097, -0.039958134, -0.014660718, 0.018230632, 0.06685944, -0.00811018, -0.033351094, -0.02147656, 0.062323928, 0.054974563, 0.013150826, 0.0055096247, -0.04749034, -0.018200824, 0.004455907, -0.02810652, 0.020164095, -0.00876558, -0.039562214, -0.0042840443, 0.009974174, 0.011345534, -0.027136857, -0.059997555, -0.020734908, 0.021480415, 0.021261424, -0.03684696, -0.03498468, 0.022974245, -0.044404, 0.056523673, -0.030762417, 0.007820478, -0.039046016, -0.04396741, -0.030224098, -0.049853057, 0.0012070368, -0.0076637506, 0.008231041, -0.083567224, -0.010287503, 0.029966535, -0.004241042, -0.020828733, 0.043242592, 0.043772276, 0.019570338, -0.05524451, 0.02760254, 0.028772337, 0.0658252, -0.045344286, -0.009925913, -0.006719821, 0.025545625, -0.032602057, 0.024687584, 0.0033193657, 0.0004900887, 0.024482781, -0.015778888, 0.05321495, 0.019637126, -0.059270747, -0.042733073, -0.0027987354, 0.01689259, 0.0430041, 0.007180756, 0.011045536, 0.058955528, -0.062358852,

In [9]:
DOCUMENT1 = {
    "title": "Operating the Climate Control System",
    "content": "Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."}
DOCUMENT2 = {
    "title": "Touchscreen",
    "content": "Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the \"Navigation\" icon to get directions to your destination or touch the \"Music\" icon to play your favorite songs."}
DOCUMENT3 = {
    "title": "Shifting Gears",
    "content": "Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [10]:
df = pd.DataFrame(documents)
df.columns = ['Title', 'Text']
df

,Title,Text
0,Operating the Climate Control System,Your Googlecar has a climate control system th...
1,Touchscreen,Your Googlecar has a large touchscreen display...
2,Shifting Gears,Your Googlecar has an automatic transmission. ...


In [11]:
model="models/embedding-001"

In [12]:
def embedding_fn(title, text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [13]:
df["embeddings"] = df.apply(lambda row: embedding_fn(row["Title"],row["Text"]), axis=1)
df

,Title,Text,embeddings
0,Operating the Climate Control System,Your Googlecar has a climate control system th...,"[-0.033361107, -0.021217084, -0.049581926, -0...."
1,Touchscreen,Your Googlecar has a large touchscreen display...,"[0.009660736, -0.030662702, -0.017281422, -0.0..."
2,Shifting Gears,Your Googlecar has an automatic transmission. ...,"[-0.04270796, -0.007160868, -0.03242516, -0.02..."


In [17]:
def gerar_e_buscar_consulta(consulta, base, modelo):
  embedding_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")
  produtos_escalares = np.dot(np.stack(df["embeddings"]), embedding_consulta["embedding"])
  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Text"]

In [33]:
consulta = "How to shift gears in Googlecar?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions.


In [35]:
generation_config = {
    "candidate_count":1,
    "temperature":0.5
}

In [37]:
prompt = f"Rewrite this text in a more relaxed way, without adding information that is not part of the text: {trecho}"

model_2 = genai.GenerativeModel('gemini-1.0-pro', generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

Yo, your Googlecar's got an automatic, so you don't have to sweat about shifting gears. Just slide the lever to where you want to be:

* **Park:** Stop sign, yo. Your ride's chillin'.
* **Reverse:** Time to back this baby up.
* **Neutral:** Engine still running, but your ride's sitting tight.
* **Drive:** Hit the road, Jack!
* **Low:** Get a grip when it's slippery out there.
